In [ ]:
!pip install -q findspark
!pip install pyspark

import findspark
findspark.init()
from pyspark.sql import SparkSession

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 50.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=6373d6bb4c2d8aa2769afcdcd45a2009b7429ac2343e0f6d7e47b4ad737bc727
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


Task 1

Step 1: Remove the records where “user_verified” is “FALSE”.

Step 2: For the remaining records (“user_verified” is “TRUE”), group by created date, and count the number of tweets for each date. Example: If “tweet_created_at” is “Tue Nov 01 01:57:25 +0000 2016”, the created date is “Nov 01”.

Step 3: For the date with highest number of tweets (you can figure it out from step 2), calculate the sum of “favorite_count” and “retweet_count” for each tweet on that day. Then report the text content (“text_”) of the top 100 tweets with highest sum. Count the word frequency of the 100 tweets and report the result (Note that data cleaning steps before wordcount can be done outside of Spark operations).

IMPORTING THE DATA FILE

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.csv('Amazon_Responded_Oct05.csv',header = True)

Convert Data to Resilient Distributed Data 

In [ ]:
data_rdd = df.rdd

Step 1: Remove records with "user_verified" is FALSE 

In [ ]:
# Selecting the six columns needed 
df_rdd = df.select('id_str','tweet_created_at','user_verified','favorite_count','retweet_count','text_')
# Filtering the data with user_verified
df_rdd_new = df_rdd.filter(df_rdd.user_verified=='True')

users_verified = df_rdd_new.rdd
users_verified.count()

80201

Step 2: Count the number of tweets for each created date

In [ ]:
def joiningtweets(line):
      ln = line['tweet_created_at'].split(":")[0][4:10]
      return ln

#Getting the dates by concatinating and counting them 
dates = users_verified.map(joiningtweets)
dates_count = dates.countByValue().items()
print(dates_count)


dict_items([('Nov 01', 370), ('Nov 02', 876), ('Nov 06', 357), ('Nov 04', 355), ('Nov 08', 519), ('Nov 05', 1058), ('Nov 03', 440), ('Nov 07', 592), ('Nov 09', 231), ('Nov 12', 720), ('Nov 10', 93), ('Nov 11', 149), ('Nov 13', 226), ('Nov 14', 349), ('Nov 15', 318), ('Nov 16', 104), ('Nov 18', 1246), ('Nov 17', 101), ('Nov 19', 457), ('Nov 20', 789), ('Nov 22', 274), ('Nov 21', 213), ('Nov 23', 907), ('Nov 25', 373), ('Nov 26', 521), ('Nov 24', 593), ('Nov 28', 270), ('Nov 27', 208), ('Nov 29', 952), ('Dec 01', 875), ('Dec 03', 1197), ('Nov 30', 278), ('Dec 04', 884), ('Dec 02', 314), ('Dec 05', 824), ('Dec 07', 360), ('Dec 08', 125), ('Dec 06', 767), ('Dec 09', 431), ('Dec 13', 979), ('Dec 11', 918), ('Dec 12', 372), ('Dec 10', 979), ('Dec 15', 1276), ('Dec 14', 270), ('Dec 17', 221), ('Dec 16', 363), ('Dec 18', 178), ('Dec 19', 240), ('Dec 21', 819), ('Dec 23', 196), ('Dec 20', 150), ('Dec 22', 651), ('Dec 26', 978), ('Dec 24', 247), ('Dec 25', 433), ('Dec 29', 518), ('Dec 27', 442),

In [ ]:
import operator
sorted_d = sorted(dates.countByValue().items(), key=operator.itemgetter(1),reverse=True)
print(sorted_d)

[('Jan 03', 1536), ('Jan 10', 1508), ('Jan 11', 1496), ('Jan 12', 1410), ('Jan 06', 1364), ('Jan 07', 1360), ('Jan 20', 1336), ('Jan 13', 1295), ('Jan 14', 1290), ('Jan 21', 1290), ('Jan 18', 1286), ('Dec 15', 1276), ('Jan 24', 1259), ('Nov 18', 1246), ('Dec 03', 1197), ('Jan 02', 1195), ('Jan 19', 1175), ('Jan 25', 1162), ('Jan 23', 1149), ('Jan 08', 1143), ('Jan 17', 1124), ('Jan 26', 1082), ('Jan 27', 1080), ('Jan 04', 1071), ('Nov 05', 1058), ('Feb 09', 1054), ('Jan 22', 1024), ('Jan 16', 1007), ('Jan 15', 1006), ('Jan 09', 1003), ('Feb 02', 987), ('Dec 13', 979), ('Dec 10', 979), ('Dec 26', 978), ('Jan 30', 968), ('Feb 03', 960), ('Jan 28', 955), ('Nov 29', 952), ('Feb 14', 950), ('Feb 08', 926), ('Feb 15', 919), ('Dec 11', 918), ('Feb 13', 916), ('Jan 31', 912), ('Nov 23', 907), ('Feb 01', 902), ('Dec 04', 884), ('Feb 10', 880), ('Nov 02', 876), ('Dec 01', 875), ('Feb 07', 875), ('Feb 04', 869), ('Feb 11', 868), ('Dec 31', 835), ('Dec 05', 824), ('Dec 21', 819), ('Feb 06', 814), 

Step 3: Find the sum of "favourite_count" and "retweet_count"

In [ ]:
#Extracting the rows we require 
updated_dates = users_verified.map(lambda row: (row["id_str"], row["tweet_created_at"],row["user_verified"],\
                                        row["favorite_count"],row["retweet_count"],row["text_"]))
updated_dates.first()

("'793281386912354304'",
 'Tue Nov 01 02:39:55 +0000 2016',
 'True',
 '0',
 '0',
 "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL")

In [ ]:
#Creating a function to return the sum 
#Filter out the empty values and return -1. 
#If there is any sort of exception encounterd the returned value is set to -1.

def sums(x,y):
    try:
        if(x!= None and y!= None):
            return(int(x)+int(y))
        else:
            return - 1
    except ValueError:
        return -1

In [ ]:
# Filter first 100 tweets based on sum

tweets100 = updated_dates.filter(lambda row : row[1][4:10] == "Jan 03").map(lambda x : (sums(x[3],x[4]),x[5])).sortByKey(False).take(100)

In [ ]:
# Tweets in rdd
rddtweets = spark.sparkContext.parallelize(tweets100)

In [ ]:
#Punctutations to be deleted from the reviews before the word count

punctutation = ["(", "[", ",", ".", "!", "?", ":", ";", "]", ")", "@", "^",'#','â€‹hey','ðŸ˜Š','ðŸ˜Œ','ðŸ˜·ðŸ˜·','ðŸ™‹']

def del_punc(line):
    for char in line:
        if char in punctutation:
          line = line.replace(char,"")
    return line

In [ ]:
wordsreqd = rddtweets.map(lambda x : x[1]).map(lambda line : del_punc(line))\
.flatMap(lambda line : line.lower().split()).map(lambda word : (word, 1)).\
reduceByKey(lambda p,q : p+q).sortByKey(ascending = True).collect()

wordsreqd

[('&amp', 1),
 ('+', 1),
 ('-', 2),
 ('1', 1),
 ('1/2', 2),
 ('1/3', 2),
 ('2/2', 2),
 ('2/3', 1),
 ('20', 1),
 ('2100', 1),
 ('3-5', 1),
 ('3/3', 1),
 ('3/3sr', 1),
 ('36hrs', 1),
 ('8', 1),
 ('__naijadrew', 1),
 ('_dmarie99', 1),
 ('_izzyscott', 1),
 ('a', 24),
 ('a-to-z', 1),
 ('ab', 1),
 ('abhinavsaroj3', 4),
 ('able', 3),
 ('about', 4),
 ('aboutasix', 1),
 ('above', 2),
 ('access', 3),
 ('account', 2),
 ('accttypically', 1),
 ('add', 1),
 ('address', 1),
 ('adi_baddy', 1),
 ('advise', 1),
 ('ae', 3),
 ('af', 2),
 ('aim', 2),
 ('ak', 1),
 ('alerts', 2),
 ('alexa', 1),
 ('alexisgraham99', 1),
 ('allow', 1),
 ('already', 1),
 ('always', 3),
 ('alyssagoldman', 3),
 ('am', 1),
 ('amazon', 7),
 ('amazonhelp', 1),
 ('amazonin', 2),
 ('ambermullennn', 1),
 ('an', 7),
 ('and', 17),
 ('any', 8),
 ('anything', 1),
 ('apologies', 2),
 ('app', 6),
 ('application', 1),
 ('appreciation', 1),
 ('ar', 3),
 ('are', 2),
 ("aren't", 1),
 ('ariannadelbene', 1),
 ('arrange', 1),
 ('arrive', 6),
 ('arri

Task 2

You will use find_text.csv for this task. There are two columns in this document: “id_str” and “text”. The second column is empty. Please find out the text content of each tweet according to “id_str” joining Amazon_Responded_Oct05.csv and fill in the “text” column.

Note: If a tweet ID appears in multiple records, just report the text content from one of them

In [ ]:
import pandas as pd

dataset = pd.DataFrame(wordsreqd) 
dataset.columns = (['Words','Count'])
dataset
dataset.to_csv('count.csv',index = True)

In [ ]:
# Import the find_text file
dataset1 = spark.read.csv('find_text.csv',header = True) 
dataset1_rdd = dataset1.rdd 
data_amazon_rdd = df.rdd

In [ ]:
# Creating the joins
join = data_amazon_rdd.map(lambda x: (x['id_str'],x['text_'])) 

dataset1 = dataset1_rdd.map(lambda x: (x['id_str'],'1'))
tweet_display = dataset1.join(join).map(lambda x : (x[0], x[1][1])).collect()
tweet_display

[("'793322433625415680'",
  '@Murtazansp Could you please tell us if the product is fulfilled by amazon or fulfilled by a seller? ^NR (2/2)'),
 ("'793383832720474113'",
  "@praveen_pandey_ Hey! Sorry for the inconvenience. Kindly drop your details here: https://t.co/GIJyeYqKE0 and I'll look into this. ^HG"),
 ("'793397280254472193'",
  '@anchitkhar HI! We have sent the correspondence to your email, I request you to check your inbox. ^SP'),
 ("'793407430344310785'",
  '@AmazonHelp How many times do you want to write back to you guys??? Check the complaints through email regarding  Order 171-1338898-5999507.'),
 ("'793408494649081856'",
  "@mybharatraj Hi, I'm sorry but could you please elaborate your concern? So that we could look into this for you. ^AM"),
 ("'793412238073815040'",
  '@AmazonHelp due to this i didnt receive that item now ur executive ask 2 reorder it but price is less than 499 and also increase'),
 ("'793420070919766017'",
  "@bbgohel2013 Hi, I'm sorry you feel this way

In [ ]:
# Converting the new list to the dataframe

Final_Tweets = pd.DataFrame(tweet_display)
Final_Tweets.columns = (['Id_str','Text'])
Final_Tweets 

,Id_str,Text
0,'793322433625415680',@Murtazansp Could you please tell us if the pr...
1,'793383832720474113',@praveen_pandey_ Hey! Sorry for the inconvenie...
2,'793397280254472193',@anchitkhar HI! We have sent the correspondenc...
3,'793407430344310785',@AmazonHelp How many times do you want to writ...
4,'793408494649081856',"@mybharatraj Hi, I'm sorry but could you pleas..."
...,...,...
53922,'810879507192315905',@munnsey134 Please reach out to us so we can t...
53923,'810881746153639937',@AmazonHelp however iv had to buy a second rep...
53924,'810883321139970048',@munnsey134 I'm sorry for any inconvenience. I...
53925,'810900955889799168',@AmazonHelp says cant deliver to registered ad...


In [ ]:
# Convertin the final output to csv
Final_Tweets.to_csv('find_text_final.csv',index = False) 